In [12]:
'''
This is the example code of benign training and poisoned training on torchvision.datasets.DatasetFolder.
Dataset is CIFAR-10.
Attack method is BadNets.
'''


import os

import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision
from torchvision.transforms import Compose, ToTensor, PILToTensor, RandomHorizontalFlip

import core



In [13]:
train_data = torchvision.datasets.CIFAR10(root='./data/cifar10/', train=True, download=True)
tes_data = torchvision.datasets.CIFAR10(root='./data/cifar10/', train=False, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [14]:

dataset = torchvision.datasets.DatasetFolder

# image file -> cv.imread -> numpy.ndarray (H x W x C) -> ToTensor -> torch.Tensor (C x H x W) -> RandomHorizontalFlip -> torch.Tensor -> network input
transform_train = Compose([
    ToTensor(),
    RandomHorizontalFlip()
])
trainset = dataset(
    root='./data/cifar10/train/',
    loader=cv2.imread,
    extensions=('png',),
    transform=transform_train,
    target_transform=None,
    is_valid_file=None)

transform_test = Compose([
    ToTensor()
])
testset = dataset(
    root='./data/cifar10/test',
    loader=cv2.imread,
    extensions=('png',),
    transform=transform_train,
    target_transform=None,
    is_valid_file=None)

index = 44

x, y = trainset[index]
print(y)
for a in x[0]:
    for b in a:
        print("%-4.2f" % float(b), end=' ')
    print()


0
0.45 0.45 0.45 0.45 0.45 0.45 0.45 0.45 0.45 0.45 0.45 0.45 0.45 0.45 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.45 0.44 0.44 0.45 
0.45 0.45 0.45 0.45 0.45 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.46 0.45 0.45 0.45 
0.45 0.45 0.45 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.46 0.46 0.46 0.45 0.45 0.45 
0.45 0.45 0.45 0.46 0.46 0.46 0.46 0.46 0.47 0.46 0.46 0.46 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.46 0.46 0.45 0.45 0.45 
0.45 0.45 0.45 0.46 0.46 0.46 0.46 0.46 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.46 0.47 0.46 0.45 0.45 0.45 
0.45 0.45 0.45 0.46 0.46 0.46 0.46 0.46 0.46 0.46 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.47 0.54 0.60 0.55 0.49 0.45 0.45 0.45 
0.45 0.45 0.45 0.46 0.46 0.46 0.

In [15]:
'''
This is the example code of benign training and poisoned training on torchvision.datasets.DatasetFolder.
Dataset is CIFAR-10.
Attack method is BadNets.
'''


import os

import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset
import torchvision
from torchvision.transforms import Compose, ToTensor, PILToTensor, RandomHorizontalFlip

import core


dataset = torchvision.datasets.DatasetFolder

# image file -> cv.imread -> numpy.ndarray (H x W x C) -> ToTensor -> torch.Tensor (C x H x W) -> RandomHorizontalFlip -> torch.Tensor -> network input
transform_train = Compose([
    ToTensor(),
    RandomHorizontalFlip()
])
trainset = dataset(
    root='./data/cifar10/train',
    loader=cv2.imread,
    extensions=('png',),
    transform=transform_train,
    target_transform=None,
    is_valid_file=None)

transform_test = Compose([
    ToTensor()
])
testset = dataset(
    root='./data/cifar10/test',
    loader=cv2.imread,
    extensions=('png',),
    transform=transform_train,
    target_transform=None,
    is_valid_file=None)

index = 44

# x, y = trainset[index]
# print(y)
# for a in x[0]:
#     for b in a:
#         print("%-4.2f" % float(b), end=' ')
#     print()


pattern = torch.zeros((1, 32, 32), dtype=torch.uint8)
pattern[0, -3:, -3:] = 255
weight = torch.zeros((1, 32, 32), dtype=torch.float32)
weight[0, -3:, -3:] = 1.0

badnets = core.BadNets(
    train_dataset=trainset,
    test_dataset=testset,
    model=core.models.ResNet(18),
    # model=core.models.BaselineMNISTNetwork(),
    loss=nn.CrossEntropyLoss(),
    y_target=1,
    poisoned_rate=0.05,
    pattern=pattern,
    weight=weight,
    poisoned_transform_train_index=0,
    poisoned_target_transform_index=0,
    schedule=None,
    seed=666
)


In [16]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    trainset,
        batch_size=128,
                shuffle=True,
                # num_workers=16,
                drop_last=False,
                pin_memory=True,
                worker_init_fn=88
            )

In [17]:
for i,j in enumerate( train_loader):
    print(i)
    print(j)
    break


0
[tensor([[[[0.1725, 0.1882, 0.1843,  ..., 0.1882, 0.1804, 0.2157],
          [0.0000, 0.0235, 0.0157,  ..., 0.0157, 0.0078, 0.0510],
          [0.0078, 0.0353, 0.0275,  ..., 0.0196, 0.0157, 0.0588],
          ...,
          [0.3686, 0.3961, 0.3922,  ..., 0.2549, 0.0471, 0.0588],
          [0.3412, 0.3529, 0.3529,  ..., 0.3216, 0.0824, 0.0471],
          [0.4157, 0.4353, 0.4314,  ..., 0.3922, 0.2431, 0.2235]],

         [[0.1725, 0.1922, 0.1882,  ..., 0.1882, 0.1804, 0.2157],
          [0.0000, 0.0275, 0.0196,  ..., 0.0196, 0.0118, 0.0510],
          [0.0118, 0.0431, 0.0353,  ..., 0.0275, 0.0235, 0.0627],
          ...,
          [0.2471, 0.2941, 0.2902,  ..., 0.2314, 0.0549, 0.0667],
          [0.2431, 0.2667, 0.2667,  ..., 0.2784, 0.0824, 0.0510],
          [0.3490, 0.3725, 0.3647,  ..., 0.3569, 0.2353, 0.2235]],

         [[0.1647, 0.1843, 0.1804,  ..., 0.1843, 0.1765, 0.2078],
          [0.0000, 0.0078, 0.0000,  ..., 0.0000, 0.0000, 0.0431],
          [0.0000, 0.0196, 0.0157,  ...

In [18]:

poisoned_train_dataset, poisoned_test_dataset = badnets.get_poisoned_dataset()


# x, y = poisoned_train_dataset[index]
# print(y)
# for a in x[0]:
#     for b in a:
#         print("%-4.2f" % float(b), end=' ')
#     print()

# x, y = poisoned_test_dataset[index]
# print(y)
# for a in x[0]:
#     for b in a:
#         print("%-4.2f" % float(b), end=' ')
#     print()

# train benign model
schedule = {
    'device': 'GPU',
    'CUDA_VISIBLE_DEVICES': '0',
    'GPU_num': 1,

    'benign_training': True,
    'batch_size': 128,
    'num_workers': 16,

    'lr': 0.1,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'gamma': 0.1,
    'schedule': [150, 180],

    'epochs': 5,

    'log_iteration_interval': 100,
    'test_epoch_interval': 10,
    'save_epoch_interval': 10,

    'save_dir': 'experiments',
    'experiment_name': 'train_benign_DatasetFolder-CIFAR10'
}

badnets.train(schedule)


# train attacked model
schedule = {
    'device': 'GPU',
    'CUDA_VISIBLE_DEVICES': '0',
    'GPU_num': 1,

    'benign_training': False,
    'batch_size': 128,
    'num_workers': 16,

    'lr': 0.1,
    'momentum': 0.9,
    'weight_decay': 5e-4,
    'gamma': 0.1,
    'schedule': [150, 180],

    'epochs': 200,

    'log_iteration_interval': 100,
    'test_epoch_interval': 10,
    'save_epoch_interval': 10,

    'save_dir': 'experiments',
    'experiment_name': 'train_poisoned_DatasetFolder-CIFAR10'
}

badnets.train(schedule)

This machine has 1 cuda devices, and use 1 of them to train.
Total train samples: 5000
Total test samples: 10000
Batch size: 128
iteration every epoch: 39
Initial learning rate: 0.1

Epoch 1/5
